# Vault Setup

In this tutorial, we'll explore various ways of setting up a HashiCorp Vault, including running it as a local binary, as a Docker container, and via Docker Compose. This will cover both Vault OSS and Vault Enterprise.

## Pre-requisites

Before we begin, ensure you have the following software installed on your system:

- `docker`
- docker compose
- `jq` (a lightweight and flexible command-line JSON processor)
- `vault` binary

### Set Up Environment Variables

We will start by setting up a few environment variables. You can customize these variables based on your preferences and requirements.

* First, we'll create some directories for Vault to use.

In [ ]:
# Set directory environment variables.
MAIN_DIR=$(pwd)
WORK_DIR=/tmp/config/vault

# Create required directories.
mkdir -p $WORK_DIR/{data,logs,config}

# Verify the structure of the directories.
tree $WORK_DIR

- You should see the following output:
    
    ```bash
    /tmp/config/vault
    ├── config
    ├── data
    └── logs
    
    3 directories, 0 files
    ```

* Now, let's define some common environment variables that Vault will use.

In [ ]:
# Common
export VAULT_VER=1.7.5
# Ent 1.8+ requires license file; Ent 1.7.2 has 8 hour eval
export VAULT_PORT=8200
export VAULT_TOKEN=root
export VAULT_ADDR=http://localhost:${VAULT_PORT}
export VAULT_LICENSE=$(cat ../../license/vault.hclic)

# Running Vault as a Docker Container

Vault can be run in a Docker container. This method is preferable for quickly testing different versions of Vault or integrating with additional components such as Consul, web interfaces, databases, etc.

> Note: Do not try to run Vault as both a Docker container and a local binary at the same time unless you modify the ports used first.

## **Running Vault via Docker**

- To run Vault via Docker, execute the following commands:
    

In [ ]:
docker stop vault # stop any existing vault container
docker run --rm -d \
  --name vault \
  -p 8200:8200 \
  -e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN:-root}" \
  -e "VAULT_ADDR=http://127.0.0.1:8200" \
  -e "VAULT_TOKEN=${VAULT_TOKEN}" \
  --cap-add=IPC_LOCK \
  hashicorp/vault:${VAULT_VER}

## Running Vault via Docker Compose

Docker Compose is a tool for defining and running multi-container Docker applications. In this case, we will use Docker Compose to set up and run Vault.

### Configuration of Vault Container

Before we run Vault with Docker Compose, we have to configure the Docker Compose file correctly. Here are two options you need to consider and choose based on your requirements:

1. Open-Source Software (OSS) vs. Enterprise:
    - If you are using Vault OSS
        - comment out the line `image: hashicorp/vault-enterprise`
        - uncomment the line `image: hashicorp/vault`.
    - If you are using Vault Enterprise
        - uncomment the line `image: hashicorp/vault-enterprise`
        - comment out the line `image: hashicorp/vault`.
2. Development Mode vs. Local Configuration:
    - If you want to run Vault in development mode:
        - uncomment the line `entrypoint: "vault server -dev"`
        - comment out the lines `entrypoint: "vault server -config=/vault/config"` and `./config/vault/config:/vault/config:ro`.
    - If you want to run Vault with local configuration
        - comment out the line `entrypoint: "vault server -dev"`
        - uncomment the lines `entrypoint: "vault server -config=/vault/config"` and `./config/vault/config:/vault/config:ro`.
            - This requires initialization and unseal.

By default, Vault Enterprise runs in Development Mode.

- Set your options below and create a `docker-compose.yaml` file.

In [ ]:
## Set the variables. Adjust according to your preferences.
export VAULT_TYPE="oss" # Use "oss" for OSS version or "enterprise" for Enterprise version.
export VAULT_MODE="local" # Use "dev" for Dev Mode or "local" for local configuration mode.

# Determine the image and entrypoint based on the environment variables
if [ "$VAULT_TYPE" = "enterprise" ]; then
    IMAGE="hashicorp/vault-enterprise:${VAULT_VER}_ent"
else
    IMAGE="hashicorp/vault:${VAULT_VER}"
fi

if [ "$VAULT_MODE" = "dev" ]; then
    ENTRYPOINT="vault server -dev"
else
    ENTRYPOINT="vault server -config=/vault/config"
fi

## Create docker-compose file.
cat > $WORK_DIR/docker-compose.yaml << EOF
version: '3.8'
services:
  vault:
    image: $IMAGE
    container_name: vault
    restart: always
    volumes:
      - ${WORK_DIR}/logs:/vault/logs
    ports:
      - "$VAULT_PORT:8200/tcp"
    environment:
      VAULT_DEV_ROOT_TOKEN_ID: ${VAULT_TOKEN:-root}
      VAULT_DEV_LISTEN_ADDRESS: "0.0.0.0:8200"
      VAULT_ADDR: ${VAULT_ADDR}
      VAULT_LICENSE: ${VAULT_LICENSE}
    cap_add:
      - IPC_LOCK
    entrypoint: "$ENTRYPOINT"
EOF

if [ "$VAULT_MODE" = "local" ]; then
    if [[ "$(uname)" == "Darwin" ]]; then
        sed -i '' "/volumes:/a\\
      - ${WORK_DIR}/config:/vault/config:ro
" $WORK_DIR/docker-compose.yaml
    else
        sed -i "/volumes:/a \
      - ${WORK_DIR}/config:/vault/config:ro
" $WORK_DIR/docker-compose.yaml
    fi
fi

In [ ]:
cat $WORK_DIR/docker-compose.yaml

### Vault Configuration for Non-Development Mode

If you are planning to run Vault with local configuration, you need to create a Vault configuration file. This configuration file should be placed in a directory that will be mounted by the Docker container.

In [ ]:
cat > $WORK_DIR/config/vault1.hcl << EOF
# storage "file" {
#     path = "/vault/file"
# }

storage "raft" {
  path    = "./vault/file"
  node_id = "node1"
}

listener "tcp" {
  address       = "0.0.0.0:8200"
#   tls_cert_file = "/path/to/fullchain.pem"
#   tls_key_file  = "/path/to/privkey.pem"
  tls_disable = true
  telemetry {
    unauthenticated_metrics_access = true
  }
}

default_lease_ttl = "168h" # default(768h)
max_lease_ttl = "0h" # default(768h)
api_addr = "http://0.0.0.0:8200"
cluster_addr = "https://0.0.0.1:8201"
ui = true

log_level = "debug"
telemetry {
  prometheus_retention_time = "24h"
  disable_hostname = true
}
EOF

In [ ]:
cat $WORK_DIR/config/vault1.hcl

### Starting the Vault Container

* Now you can start the Vault container using Docker Compose:

In [ ]:
docker-compose -f $WORK_DIR/docker-compose.yaml up --force-recreate -d 2>&1 \
  | tee /tmp/vault_docker_compose.log

## ****Verifying the Vault Container****

To make sure that Vault is running correctly, you can check the status of Vault:

In [ ]:
vault status && echo #Via CLI
curl -s http://localhost:8200/v1/sys/seal-status | jq # Via API

The output should show that the Vault server is initialized and unsealed if you are running in dev mode. It will be sealed if not in dev mode.

---

# Running Vault Locally as Binary

In case you want to run Vault locally as a binary instead of inside a Docker container, you can do that as well.

You will need to run the next few commands from a terminal if you are on a Mac. sudo requires an interactive prompt.

First, download the appropriate Vault binary:

In [ ]:
HASHI_RELEASES=https://releases.hashicorp.com
VAULT_VER=1.10.4   # 1.8+ enterprise requires license; no timed eval
VAULT_ENT=""       #"+ent" or ""
VAULT_REL=${VAULT_VER}${VAULT_ENT}

# Automatically detect the operating system
if [[ "$(uname)" == "Darwin" ]]; then
    VAULT_OS="darwin" && echo $VAULT_OS
elif [[ "$(uname)" == "Linux" ]]; then
    VAULT_OS="linux"
else
    echo "Unsupported operating system."
fi

# Automatically detect the hardware architecture
VAULT_HW=$(uname -m)
if [[ "$VAULT_HW" == "x86_64" ]]; then
    VAULT_HW="amd64" && echo $VAULT_HW
elif [[ "$VAULT_HW" == "aarch64" ]]; then
    VAULT_HW="arm64"
elif [[ "$VAULT_HW" == "arm64" ]]; then
    VAULT_HW="arm64"
else
    echo "Unsupported hardware architecture."
fi

In [ ]:
curl -o /tmp/vault.zip \
  ${HASHI_RELEASES}/vault/${VAULT_REL}/vault_${VAULT_REL}_${VAULT_OS}_${VAULT_HW}.zip \
  && sudo unzip -o -d /usr/local/bin /tmp/vault.zip \
  && echo sudo mv -f vault /usr/local/bin \
  && rm /tmp/vault.zip

Verify that the local binary works.

In [ ]:
/usr/local/bin/vault version

Then, you can start the Vault server:

In [ ]:
/usr/local/bin/vault server \
  -dev -dev-root-token-id=${VAULT_TOKEN} \
  -dev-listen-address=0.0.0.0:8200 \
  -log-level=trace \
  -config=./config.hcl > ${WORK_DIR}/vault.log 2>&1 &

Check the Vault logs to see if everything is running smoothly:

In [ ]:
head ${WORK_DIR}/vault.log
tail ${WORK_DIR}/vault.log

In [ ]:
vault status

---

# Initialize and Unseal Vault

In non-development mode, Vault starts up sealed and you have to initialize it before you can unseal and use it.

## Initialize Vault

In [ ]:
vault operator init -key-shares=1 -key-threshold=1 > $WORK_DIR/vault.init
cat $WORK_DIR/vault.init

This command initializes the vault server and generates master keys and the initial root token. It also provides key shards that you can distribute to trusted users.

The `-key-shares` and `-key-threshold` options are used to control the number of key shards and the number of key shards required to unseal Vault, respectively. For this tutorial, we keep things simple by setting both values to `1`.

The output of the `vault operator init` command is redirected to a `vault.init` file in the `WORK_DIR` directory.

### Extract Root Token and Unseal Key

In [ ]:
VAULT_TOKEN=$(grep "Root Token" $WORK_DIR/vault.init | awk '{print $NF}') && echo $VAULT_TOKEN
UNSEAL_KEY1=$(grep "Key 1" $WORK_DIR/vault.init | awk '{print $NF}') && echo $UNSEAL_KEY1

The root token and unseal key are extracted from the `vault.init` file and stored in `VAULT_TOKEN` and `UNSEAL_KEY1` variables, respectively.

## Unseal Vault

In [ ]:
vault operator unseal ${UNSEAL_KEY1}

Vault uses an unseal process to ensure that secrets are securely protected when the server restarts. Unsealing is the process of constructing the master key necessary to read the decryption key to decrypt the data, allowing access to the Vault.

# Status Check, Audit Enablement, and Licensing

This section describes how to check the status of Vault, enable audit logging, and how to install a license for Vault Enterprise.

## Check Vault Status

In [ ]:
vault status

Running the `vault status` command shows you the status of Vault. After the unseal operation, Vault should show the status as "Unsealed".

## Enable Audit Logging

Audit logs record the requests and responses to Vault. All interactions with Vault are logged in an audit log to keep a detailed audit trail.

- Enable audit output to files with and without hashing. Do not leave `log_raw` on in production.

In [ ]:
# Enable audit output to file, audit settings are in vault1.hcl
vault audit enable file file_path=/vault/logs/vault_audit.log
vault audit enable file file_path=/tmp/vault_audit.log #works in container or localhost
vault audit enable -path=raw file file_path=/vault/logs/audit_raw.log log_raw=true

- The `vault audit enable` command enables an audit device at a given path.

In [ ]:
vault audit list

## License Installation (for Vault Enterprise)

Install license via API. Required for Vault Enterprise prior to 1.8.

In [ ]:
curl \
    --silent \
    --header "X-Vault-Token: $VAULT_TOKEN" \
    --request PUT \
    --data '{"text":"'$VAULT_LICENSE'"}' \
    $VAULT_ADDR/v1/sys/license

This command installs the license for Vault Enterprise using the curl utility to send an HTTP PUT request to Vault's `/sys/license` endpoint.

### Verify Status Again

Run the `vault status` command again to confirm Vault is still unsealed after the license has been applied.

In [ ]:
vault status

## Access Vault's Web Interface (UI

Vault provides a web interface (UI) that you can use to interact with Vault.

## UI

In [ ]:
if [[ $(uname) == "Darwin" ]]; then
  open $VAULT_ADDR
else
  printf "\nGo to http://127.0.0.1:8200\n"
fi
printf "\n%s\n" "-----------------------------------"
printf "\n${GREEN}Login with token:${NC} $VAULT_TOKEN \n"
printf "\n%s\n" "-----------------------------------"

This script opens the Vault web interface in your default web browser. If you're using a Linux operating system, you need to manually open a web browser and navigate to the Vault web interface at [http://127.0.0.1:8200](http://127.0.0.1:8200/).

You can login with the root token that you have saved in the `VAULT_TOKEN` environment variable. After logging in, you should be able to interact with Vault through the web interface.

## Sample Authentication Methods

* For LDAP configuration, see [LDAP](./110-Setup-authmethods.ipynb#LDAP)
* For User Pass, see [UserPass](./110-Setup-authmethods.ipynb#UserPass)

## Clean Up

After you're done using your Vault setup for the purpose of this guide, you might want to clean everything up. Here's how you can do that:

## 1. Seal Vault

First, you should seal Vault. This ensures that all the data stored in Vault is encrypted and Vault stops accepting any more API calls.

You can seal Vault by running:

In [ ]:
vault operator seal

## 2. Stop and Remove Docker Container or Binary

Next, depending on your setup:

### If you're running Vault inside a Docker container

To stop the Docker container, use the `stop` command:

In [ ]:
docker stop vault

And to remove the container:

In [ ]:
docker rm vault

### If you're running Vault using Docker Compose:

To stop and remove the container, networks, volumes, and images defined by your Docker Compose file, use the `down` command:

In [ ]:
docker-compose -f /tmp/config/vault/docker-compose.yaml down
docker ps | grep -i vault

### If you're running Vault as a local binary:

To stop the Vault binary running on your localhost, you can use `pkill`:

In [ ]:
pkill vault

## 4. Clean up artifacts

Finally, you might want to remove any data files generated by Vault and Docker. The specifics of this step will depend on your setup, but in general, you should remove any data files or directories that were created as part of running Vault. Make sure you only delete files and directories you know were created as part of this process to avoid losing important data.

In [ ]:
rm -rf /tmp/config/vault/*
rm /tmp/vault_audit.log /tmp/vault_docker_compose.log
rm -rf /tmp/config/vault/docker-compose.yaml